# Alpha by NLP: Defining functions to get stock returns
In this notebook, we will define two functions:  
`getRet(symbol, date, horizon, subtract)` and `getTotalRet(symbol,date,horizon, subtract)`.

Suppose we have a .csv file with schema  
`   0      1      2      3      4      5      6      7      8      9      10`  
`Date   Symbol Return                                    Alpha`

`getRet()` returns a list of daily returns `[.05, .03, -.01, -.01, .02]`

`getTotalRet()` returns the total return, i.e. the return you get if you put in $1 at the start date and sold at the end date `(start date + horizon - 1)` 
* Note that `(1+avg(rets)) ** n != getTotalRet(n)` where n is the number of days

In [2]:
import csv
from datetime import datetime, timedelta
import numpy as np
import os.path

dirname = 'data'
dirname2 = 'Stock_Returns'
dateformat = '%Y-%M-%d'

def getRets(symbol, date, horizon=1, subtract=False):
    """
    Assumed csv schema is:
    0      1      2      3      4      5      6      7      8      9      10
    Date   Symbol Return                                    Alpha
    """
    some_dates = []
    the_date1 = datetime.strptime(date, dateformat)
    for i in range(horizon):
        the_date2 = the_date1 + timedelta(days=i)
        some_dates.append(the_date2)
        
    with open(os.path.join(dirname, dirname2, 'Stocks_Plus_Alphas.csv'), 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        matching_rows = []
        for row in reader:
            if(row[0] == 'Date'):
                pass
            else:
                date1 = datetime.strptime(row[0], dateformat)
                if (date1 in some_dates and row[1] == symbol):
                    if (subtract):
                        matching_rows.append([row[0], row[1], row[8]])
                    else:
                        matching_rows.append([row[0], row[1], row[2]])
    return matching_rows

def getTotalRet(symbol, date, horizon=1, subtract=False):
    rets = getRets(symbol, date, horizon, subtract)
    the_rets = []
    for ret in rets:
        the_rets.append(ret[2])
    return np.product([1 + float(the_ret) for the_ret in the_rets])-1

## Getting a list of returns for a stock
Let's try to get a list of returns starting at 8/19/99 for BBY (Blackberry) and ending 5 days later -  
Observe that weekends and holidays are omitted.

In [3]:
import time
start = time.clock()

ans = getRets('BBY', '1999-08-19', 5)

end = time.clock()
print ("Total running time: ", end-start)
print(ans)

Total running time:  122.63690700000001
[['1999-08-19', 'BBY', '-0.0135379061372'], ['1999-08-20', 'BBY', '0.0631290027447'], ['1999-08-23', 'BBY', '-0.0275387263339']]


## Getting total return for a stock
Now, let's see our total return for BBY if we opened a position on 8/19/99 and closed it 5 days later.

In [4]:
start = time.clock()

ans = getTotalRet('BBY', '1999-08-19', 5)

end = time.clock()
print ("Total running time: ", end-start)
print(ans)

Total running time:  121.06313000000002
0.0198555956678


## Getting a list of alphas for a stock
Note that `return_stock = beta * return_S&P500 + alpha`  
For more information, refer to <a href="http://www.investopedia.com/terms/c/capm.asp">Capital Asset Pricing Model (CAPM)</a> theory.  
Let's get a list of alphas for BBY.

In [5]:
start = time.clock()

ans = getRets('BBY', '1999-08-19', 5, subtract=True)

end = time.clock()
print ("Total running time: ", end-start)
print(ans)

Total running time:  129.63147700000002
[['1999-08-19', 'BBY', '-0.00441569637804'], ['1999-08-20', 'BBY', '0.0518291598365'], ['1999-08-23', 'BBY', '-0.0488717576021']]


## Getting total cumulative alpha for stock
Note: this value is not necessarily meaningful.
Just for fun, let's get "total cumulative alpha" for a stock -

In [6]:
start = time.clock()

ans = getTotalRet('BBY', '1999-08-19', 5, subtract=True)

end = time.clock()
print ("Total running time: ", end-start)
print(ans)

Total running time:  122.89870100000002
-0.00399315039018
